# 通道套利策略：Donchain Channel

![](https://www.dropbox.com/s/8yx9cadj4ou9a4u/donchain_channel.png?dl=1)

股票市場的行情是隨機而起，一個價格的震盪到底隱含了趨勢或是雜訊？通道模型適度的解決了這個問題。它利用過去一段時間内的價格訊息，繪製出上下兩條通道綫(上、下界)，借此設定股價的相對高低界限。
Donchain Channel 流行於上個世紀 70 年代，由著名海龜交易員 Richard Donchain 發明，規則如下：

- 每天取過去 20 日的最大收盤價為上界
- 每天取過去 20 日的最小收盤價為下界
- 利用每天的上界與下界的平均值當作中界

1. 若收盤價大於過去 20 日收盤價的最大值，代表不正常上漲，所以就放空股票，直到回到中心點平倉

2. 若收盤價小於過去 20 日收盤價的最小值，代表不正常下跌，所以就做多股票，直到回到中心點平倉

In [1]:
import xlwings as xw
import numpy as np
import time

# 打開你的 tsmc_back_test.xlsx 檔案
wb = xw.Book(r'C:\Users\user\Desktop\L5\donchian_channel.xlsm')

tsmc_sheet = wb.sheets['2330']

In [2]:
# xlwings 會把 excel一個範圍的值以一個清單表示
tsmc_prices = tsmc_sheet.range("B2").options(np.array, expand="down").value
tsmc_prices

array([1455.22, 1399.42, 1402.11, ...,  869.42,  890.64,  903.25])

In [3]:
tsmc_prices.size

2263

In [4]:
lower_bound = []
# 從第 20 日開始，一次取得 20 筆資料
for i in range(19, tsmc_prices.size):
    # 利用 array slicing
    prices20 = tsmc_prices[i-19:i]
    lower_bound.append(min(prices20))


In [6]:
lower_bound_ary = np.array(lower_bound)

tsmc_sheet.range("C21").value = lower_bound_ary.reshape(lower_bound_ary.size, 1)

In [7]:
upper_bound = []
# 從第 20 日開始，一次取得 20 筆資料
for i in range(19, tsmc_prices.size):
    # 利用 array slicing
    prices20 = tsmc_prices[i-19:i]
    upper_bound.append(max(prices20))

    
upper_bound_ary = np.array(upper_bound)

tsmc_sheet.range("D21").value = upper_bound_ary.reshape(upper_bound_ary.size, 1)

In [8]:
mean_bound = []

# 從第 20 日開始，一次取得 20 筆資料
for i in range(0, upper_bound_ary.size):
    # 利用 array slicing
    mean = (upper_bound_ary[i] + lower_bound_ary[i]) / 2
    mean_bound.append(mean)
    
mean_bound_ary = np.array(mean_bound)

tsmc_sheet.range("E21").value = mean_bound_ary.reshape(mean_bound_ary.size, 1)

In [9]:
lower_bound_ary

array([1360.16, 1360.16, 1360.16, ...,  816.21,  845.22,  845.22])

In [10]:
# 實作 Bollinger band 交易 (Array 版)

# 截取出從第 20 天開始的股價
tsmc_prices_trading = tsmc_prices[19:]
# 目前是否已做多或做空的訊號
hold_flag = False
# 買入股票的訊號
long_flag = False
# 做空股票的訊號
short_flag = False
# 記錄總收益
balance = 0

trading_results = []

for i in range(0, len(tsmc_prices_trading)):
    # 截取每一天的價格、下限、平均、與上限
    price = tsmc_prices_trading[i]
    minimum = lower_bound_ary[i]
    mean = mean_bound_ary[i]
    maximum = upper_bound_ary[i]
    result = []
    print("price: {}, max:{}, min:{}, mean:{}".format(price, maximum, minimum, mean))

    # 若目前沒有進行交易
    if hold_flag == False:
        # 價格低於下限
        if price < minimum:
            # 顯示做多
            trading_results.append("Long")
            # 計算總收益
            balance = balance - (price * 1000)
            # 花費現金購買股票
            trading_results.append(balance)
            # 開啓交易訊號
            long_flag = True
            hold_flag = True
        # 價格高於上限
        elif price > maximum:
            # 顯示做空
            trading_results.append("Short")
            # 做空，現金增加
            balance = balance + (price * 1000)
            trading_results.append(balance)
            # 開啓交易訊號
            short_flag = True
            hold_flag = True
        else:
            trading_results.append("")
            trading_results.append(balance)
    # 若目前有進行交易
    elif hold_flag == True:
       # 若現在是做多，而且價格大於等於平均值，平倉
       if long_flag == True and price >= mean:
            trading_results.append("Offset")
            # 做多在平倉時，現金增加
            balance = balance + price * 1000
            trading_results.append(balance)
            hold_flag = False
            long_flag = False
       # 若現在是做空，而且價格小於等於平均值，平倉
       elif short_flag == True and price <= mean:
            trading_results.append("Offset")
            # 做空在平倉時，現金減少
            balance = balance - price * 1000
            trading_results.append(balance)
            hold_flag = False
            short_flag = False
       else:
            trading_results.append("")
            trading_results.append(balance)

# 把 ["long", 123, 0, 0, "short", 456, ...] 的清單轉成一個 n/2 x 2 的 numpy array
trading_results = np.array(trading_results).reshape(int(len(trading_results)/2), 2)
# 最後再一次性的寫入 Excel，只需要指定範圍的起點 (左上角) 即可
tsmc_sheet.range('F21').value = trading_results

# 將最後的總收益顯示在 J5
tsmc_sheet.range('J5').value = balance

price: 1394.46, max:1465.15, min:1360.16, mean:1412.6550000000002
price: 1409.28, max:1465.15, min:1360.16, mean:1412.6550000000002
price: 1409.12, max:1465.15, min:1360.16, mean:1412.6550000000002
price: 1424.97, max:1465.15, min:1360.16, mean:1412.6550000000002
price: 1424.37, max:1465.15, min:1360.16, mean:1412.6550000000002
price: 1424.24, max:1465.15, min:1360.16, mean:1412.6550000000002
price: 1441.72, max:1465.15, min:1360.16, mean:1412.6550000000002
price: 1411.71, max:1465.15, min:1360.16, mean:1412.6550000000002
price: 1416.83, max:1465.15, min:1360.16, mean:1412.6550000000002
price: 1387.12, max:1465.15, min:1360.16, mean:1412.6550000000002
price: 1389.94, max:1455.9, min:1360.16, mean:1408.0300000000002
price: 1402.05, max:1455.9, min:1360.16, mean:1408.0300000000002
price: 1387.67, max:1445.57, min:1360.16, mean:1402.865
price: 1388.26, max:1441.72, min:1360.16, mean:1400.94
price: 1346.09, max:1441.72, min:1360.16, mean:1400.94
price: 1352.17, max:1441.72, min:1346.09, me

# Python 的資料科學套件：Pandas

提供靈活直觀的資料結構來處理關聯數據和有標籤的數據

---
## Pandas 資料結構

| 名稱 | 描述 |
|:--:|:------:|
| Series | 可以建立索引的一維陣列       |
| DataFrame	 | 有列索引與欄標籤的二維資料集 |
| Panel | 有資料集索引、列索引與欄標籤的三維資料集 |

---
## DataFrame (資料框架)
- 表格型資料結構 (可以想像成一個虛擬的 Excel 試算表)
- 實際上是由多個 Series 組合起來的資料結構
- 適用於二維的資料

---
## .options(format, expand=)

- **format** 是你希望 xlwings 將一個範圍的值以**什麽樣的資料結構回傳**
- **expand** 是讓 xlwings **自動偵測試算表資料的範圍**，"down" 代表往下搜尋，"table" 則是將整個試算表，連續的資料一次性的搜出

## 參考文件：

[Pandas 官方文件](https://pandas.pydata.org/pandas-docs/stable/10min.html)

In [12]:
import pandas as pd

# xlwings 會把 excel一個範圍的值以 pandas DataFrame 的形式回傳
tsmc_prices_df = tsmc_sheet.range("A1").options(pd.DataFrame, expand="table").value
tsmc_prices_df

,Closing,Lower,Upper,Mean,Signal,Balance
Date,,,,,,
2000-01-03,1455.22,None,None,None,None,None
2000-01-04,1399.42,None,None,None,None,None
2000-01-05,1402.11,None,None,None,None,None
2000-01-06,1403.45,None,None,None,None,None
2000-01-07,1441.47,None,None,None,None,None
2000-01-10,1457.60,None,None,None,None,None
2000-01-11,1438.56,None,None,None,None,None
2000-01-12,1432.25,None,None,None,None,None
2000-01-13,1449.68,None,None,None,None,None


In [25]:
# 計算出 20 日最大與最小值
max_ary = [max(tsmc_prices_df["Closing"][i-19:i]) for i in range(19, len(tsmc_prices_df["Closing"]))]
min_ary = [min(tsmc_prices_df["Closing"][i-19:i]) for i in range(19, len(tsmc_prices_df["Closing"]))]

In [26]:
# 截取 20 日以後的資料
tsmc_data_df = tsmc_prices_df[19:]

In [27]:
# 語法非常直覺
tsmc_data_df["Lower"] = min_ary
tsmc_data_df["Upper"] = max_ary

tsmc_data_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Closing,Lower,Upper,Mean,Signal,Balance
Date,,,,,,
2000-01-31,1394.46,1360.16,1465.15,None,None,None
2000-02-01,1409.28,1360.16,1465.15,None,None,None
2000-02-02,1409.12,1360.16,1465.15,None,None,None
2000-02-03,1424.97,1360.16,1465.15,None,None,None
2000-02-04,1424.37,1360.16,1465.15,None,None,None
2000-02-07,1424.24,1360.16,1465.15,None,None,None
2000-02-08,1441.72,1360.16,1465.15,None,None,None
2000-02-09,1411.71,1360.16,1465.15,None,None,None
2000-02-10,1416.83,1360.16,1465.15,None,None,None


In [28]:
# 用 DataFrame 現有的資料計算出新的一欄資料
tsmc_data_df["Mean"] = (tsmc_data_df["Upper"] + tsmc_data_df["Lower"]) * 0.5
tsmc_data_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Closing,Lower,Upper,Mean,Signal,Balance
Date,,,,,,
2000-01-31,1394.46,1360.16,1465.15,1412.655,None,None
2000-02-01,1409.28,1360.16,1465.15,1412.655,None,None
2000-02-02,1409.12,1360.16,1465.15,1412.655,None,None
2000-02-03,1424.97,1360.16,1465.15,1412.655,None,None
2000-02-04,1424.37,1360.16,1465.15,1412.655,None,None
2000-02-07,1424.24,1360.16,1465.15,1412.655,None,None
2000-02-08,1441.72,1360.16,1465.15,1412.655,None,None
2000-02-09,1411.71,1360.16,1465.15,1412.655,None,None
2000-02-10,1416.83,1360.16,1465.15,1412.655,None,None


In [29]:
# 交易訊號目前先用舊方法處一理，截取 trading_results 第一欄的資料
signals = trading_results[:, 0]

tsmc_data_df["Signal"] = signals
tsmc_data_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Closing,Lower,Upper,Mean,Signal,Balance
Date,,,,,,
2000-01-31,1394.46,1360.16,1465.15,1412.655,,None
2000-02-01,1409.28,1360.16,1465.15,1412.655,,None
2000-02-02,1409.12,1360.16,1465.15,1412.655,,None
2000-02-03,1424.97,1360.16,1465.15,1412.655,,None
2000-02-04,1424.37,1360.16,1465.15,1412.655,,None
2000-02-07,1424.24,1360.16,1465.15,1412.655,,None
2000-02-08,1441.72,1360.16,1465.15,1412.655,,None
2000-02-09,1411.71,1360.16,1465.15,1412.655,,None
2000-02-10,1416.83,1360.16,1465.15,1412.655,,None


In [30]:
# 收益截取 trading_results 第二欄的資料
balance = trading_results[:, 1]

tsmc_data_df["Balance"] = balance
tsmc_data_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Closing,Lower,Upper,Mean,Signal,Balance
Date,,,,,,
2000-01-31,1394.46,1360.16,1465.15,1412.655,,0
2000-02-01,1409.28,1360.16,1465.15,1412.655,,0
2000-02-02,1409.12,1360.16,1465.15,1412.655,,0
2000-02-03,1424.97,1360.16,1465.15,1412.655,,0
2000-02-04,1424.37,1360.16,1465.15,1412.655,,0
2000-02-07,1424.24,1360.16,1465.15,1412.655,,0
2000-02-08,1441.72,1360.16,1465.15,1412.655,,0
2000-02-09,1411.71,1360.16,1465.15,1412.655,,0
2000-02-10,1416.83,1360.16,1465.15,1412.655,,0


In [31]:
result_sheet = wb.sheets["results"]

In [32]:
result_sheet.range("A1").value = tsmc_data_df

In [33]:
import matplotlib.pyplot as plt
fig = plt.figure()
# 截取 DataFrame 内某一個時段的資料，畫出圖
df_graph = tsmc_data_df["2000/1/31":"2000/3/1"].plot()

In [34]:
figure = df_graph.get_figure()
plot = result_sheet.pictures.add(figure,name="Donchain Channel", left=result_sheet.range('H1').left, top=result_sheet.range('H1').top)

In [35]:
plot.height *= 2
plot.width *= 2